# [LAB-08] 메일링 리스트/ DB 연동1

## #2. 메일링 리스트 구현


### 메일 발송 모듈 제작

### 메일 발송 준비

In [2]:
from mylibrary import MyMailer
import datetime as dt

today = dt.datetime.now

today = dt.datetime.now()
year = today.year
month = today.month
day = today.day
print(year, month, day)
# -----------------------------------------------------

from_addr = "운영지원팀<ssoo.alter@gmail.com>"
subjectTmpl = "{name}님의 {yy}년 {mm}월 급여명세서 입니다."

with open('mail/content.txt', 'r', encoding='utf-8') as f:
    contentTmpl = f.read()
    print(contentTmpl)

# ---------------------------------------------------------
with open('mail/mail_list.csv', 'r', encoding='euc-kr', newline='') as f:
    csv = f.readlines()
    for line in csv:
        name, email, file1, file2 = line.strip().split(",")
        #print(name, email, file1, file2)
        
        to_addr = "{name} <{email}>" .format(name=name, email=email)
        #print(to_addr)

        subject = subjectTmpl.format(name=name, yy=year, mm=month)
        #print(subject)

        content = contentTmpl.format(name=name, yy=year, mm=month, dd=day)
        #print(content)
        
        MyMailer.sendMail(from_addr,to_addr, subject,content, [file1, file2])


2025 10 29
안녕하세요 {name}님

{yy}년도 {mm}월 급여명세서와 결산보고서 보내드립니다.

귀하의 노고에 감사드립니다.

- {yy}년 {mm}월 {dd}일 / 운영지원팀 드림
document.pptx (이)가 첨부되었습니다.
pay1.xlsx (이)가 첨부되었습니다.
document.pptx (이)가 첨부되었습니다.
pay2.xlsx (이)가 첨부되었습니다.
document.pptx (이)가 첨부되었습니다.
pay3.xlsx (이)가 첨부되었습니다.
document.pptx (이)가 첨부되었습니다.
pay4.xlsx (이)가 첨부되었습니다.
document.pptx (이)가 첨부되었습니다.
pay5.xlsx (이)가 첨부되었습니다.


## #3. 비동기 처리

### 필요한 패키지 참조

In [ ]:
# 프로그램에 딜레이를 적용하는 기능을 제공하는 모듈
import time
# 날짜 처리 모듈
import datetime as dt
#앞 단원에서 구현한 메일 발송 모듈
from mylibrary import MyMailer
#비동기 처리 기능을 제공하는 모듈
import concurrent.futures as futures

### 동기식 메일 발송

In [1]:
import time
import datetime as dt
from mylibrary import MyMailer
import concurrent.futures as futures

today = dt.datetime.now()
year = today.year
month = today.month
day = today.day

from_addr = "운영지원팀<ssoo.alter@gmail.com>"
subjectTmpl = "{name}님의 {yy}년 {mm}월 급여명세서 입니다."

with open('mail/content.txt', 'r', encoding='utf-8') as f:
    contentTmpl = f.read()

startTime = dt.datetime.now()

with open('mail/mail_list.csv', 'r', encoding='euc-kr', newline='') as f:
    csv = f.readlines()

    with futures.ThreadPoolExecutor(max_workers=10) as executor:
        for line in csv:
            name, email, file1, file2 = line.strip().split(",")
            to_addr = "{name} <{email}> " .format(name=name, email=email)
            subject = subjectTmpl.format(name=name, yy=year, mm=month)
            content = contentTmpl.format(name=name, yy=year, mm=month, dd=day)
        
            executor.submit(MyMailer.sendMail, from_addr,to_addr, subject,content, [file1, file2])
endTime = dt.datetime.now()
workTime = endTime - startTime
print("작업에 소요된 시간은 총 %s초 입니다," % workTime.seconds)


document.pptx (이)가 첨부되었습니다.
document.pptx (이)가 첨부되었습니다.
document.pptx (이)가 첨부되었습니다.
document.pptx (이)가 첨부되었습니다.
document.pptx (이)가 첨부되었습니다.
pay4.xlsx (이)가 첨부되었습니다.
pay3.xlsx (이)가 첨부되었습니다.
pay2.xlsx (이)가 첨부되었습니다.
pay1.xlsx (이)가 첨부되었습니다.
pay5.xlsx (이)가 첨부되었습니다.
작업에 소요된 시간은 총 6초 입니다,


## #4. 비동기 리턴/예외처리

### 비동기로 호출되는 함수의 리턴값 받기

In [ ]:
import time
import datetime as dt
from mylibrary import MyMailer
import concurrent.futures as futures  # 비동기 처리 기능ㅇ르 제공하는 모듈

import random
def randomwork(name):
    randomSecond = random.randrange(1,9)
    print("[%s]작업을 %d초 동안 수행합니다." %(name, randomSecond))
    for i in range(0,randomSecond):
        time.sleep(1)
        print("[%s] %d초..." %(name, i+1))
    print("[%s] 작업이 종료되었습니다." %name)
    return randomSecond    #몇 초 동안 작업을 했는지 리턴

startTime = dt.datetime.now()
names = ['Lee', 'Kim', 'Hong', 'Park', 'Nam']
processes = []             #비동기 프로세스를 저장할 리스트
resultSet = []             #비동기 작업의 결과를 저장할 빈 리스트

with futures.ThreadPoolExecutor(max_workers=3) as executor:
    for n in names:
        pro = executor.submit(randomwork,n)
        processes.append(pro)
    for p in processes:             #생성도니 비동기 작업 수 만큼 반복
        result = p.result()         #randomwork 함수의 리턴값을 받음
        resultSet.append(result)        #전달받은 리턴값을 resultSet에 추가
# randomWork 함수가 리턴한 모든 값을 합산
print("비동기 처리가 총 %d초의 작업을 수행하였습니다." %sum(resultSet))

# 프로그램 실행 시간을 구함
endTime = dt.datetime.now()
workTime = endTime - startTime
print("작업에 소요된 시간은 총 %s초입니다." %workTime.seconds)

### 비동기의 예외처리

In [ ]:
import time
import datetime as dt
from mylibrary import MyMailer
import concurrent.futures as futures

import random
seasons = ['봄', '여름', '가을', '겨울']
def seasonWork(name):
    randomSecond = random.randrange(1,9)
    print("[%s] 작업을 %d초 동안 수행합니다."%(name, randomSecond))
    for i in range(0,randomSecond):
        time.sleep(1)
        # randomSecond가 4이상이면 i가 리스트의 인덱스를 초과하는 순간 에러 발생
        print("[%s] 지금은 <%s>입니다," %(name, seasons[i]))
    print("[%s] 작업이 종료되었습니다." %name)
    # 파라미터로 전달된 name과 몇 초 동안 작업을 했는지르 리스트로 리턴
    return [name, randomSecond]

startTime = dt.datetime.now()

names = ['Lee', 'Kim', 'Hong', 'Park', 'Nam']
processes = []
nameSet = []
timeSet = []

with futures.ThreadPoolExecutor(max_workers=len(names)) as executor:
    for n in names:
        pro = executor.submit(seasonWork,n)
        processes.append(pro)
    for p in processes:
        try:
            name, n = p.result()
            nameSet.append(name)
            timeSet.append(n)
            print("%s의 작업이 완료되었습니다." %name)
        except Exception as e:
            print(e)

endTime = dt.datetime.now()
workTime = endTime -startTime
print("작업에 소요된 시간은 총 %s초 입니다." % workTime.seconds)